In [129]:
import math
import eikon as ek  # the Eikon Python wrapper package
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
import configparser as cp
import scipy.optimize as sco  # optimization routines

In [130]:
cfg = cp.ConfigParser()
cfg.read('eikon.cfg')  # adjust for different file location
ek.set_app_key(cfg['eikon']['app_id']) #set_app_id function being deprecated
cf.set_config_file(offline=True)  # set the plotting mode to offline

In [131]:
rics = [
    'AAPL.O',  # Apple stock
    'AMZN.O',  # Amazon stock
    'SPY',  # S&P 500 ETF
    'GLD',  # Gold ETF
    'EUR=',  # EUR/USD exchange rate
]

In [132]:
data = ek.get_timeseries(rics,  # the RICs
                         fields='CLOSE',  # the required fields
                         start_date='2017-01-01',  # start date
                         end_date='2018-02-16')  # end date
data.head()

CLOSE,AAPL.O,AMZN.O,SPY,GLD,EUR=
Date,,,,,
2017-01-02,NaN,NaN,NaN,NaN,1.0457
2017-01-03,116.15,753.67,225.24,110.47,1.0404
2017-01-04,116.02,757.18,226.58,110.86,1.0486
2017-01-05,116.61,780.45,226.40,112.58,1.0603
2017-01-06,117.91,795.99,227.21,111.75,1.0530


In [133]:
data.dropna(inplace=True)  # deletes tows with NaN values

In [134]:
data.head()

CLOSE,AAPL.O,AMZN.O,SPY,GLD,EUR=
Date,,,,,
2017-01-03,116.15,753.67,225.24,110.47,1.0404
2017-01-04,116.02,757.18,226.58,110.86,1.0486
2017-01-05,116.61,780.45,226.40,112.58,1.0603
2017-01-06,117.91,795.99,227.21,111.75,1.0530
2017-01-09,118.99,796.92,226.46,112.67,1.0572


In [135]:
data.normalize().iplot(kind='lines')

In [136]:
# calculating daily log returns
(data / data.shift(1)).head()

CLOSE,AAPL.O,AMZN.O,SPY,GLD,EUR=
Date,,,,,
2017-01-03,NaN,NaN,NaN,NaN,NaN
2017-01-04,0.998881,1.004657,1.005949,1.003530,1.007882
2017-01-05,1.005085,1.030732,0.999206,1.015515,1.011158
2017-01-06,1.011148,1.019912,1.003578,0.992627,0.993115
2017-01-09,1.009160,1.001168,0.996699,1.008233,1.003989


In [137]:
# negative is less than 1, positive greater than 1
rets = np.log(data / data.shift(1))
rets.head()

CLOSE,AAPL.O,AMZN.O,SPY,GLD,EUR=
Date,,,,,
2017-01-03,NaN,NaN,NaN,NaN,NaN
2017-01-04,-0.001120,0.004646,0.005932,0.003524,0.007851
2017-01-05,0.005072,0.030270,-0.000795,0.015396,0.011096
2017-01-06,0.011087,0.019716,0.003571,-0.007400,-0.006909
2017-01-09,0.009118,0.001168,-0.003306,0.008199,0.003981


In [138]:
# there is a sub-plot bug
rets.iplot(kind='histogram')

In [139]:
# apple and amazon have best returns
rets.mean()  # daily mean returns used to approximate the expected returns.

CLOSE
AAPL.O    0.001396
AMZN.O    0.002309
SPY       0.000681
GLD       0.000519
EUR=      0.000621
dtype: float64

In [140]:
rets.mean() * 252 # number of trading days, annulized mean returns

CLOSE
AAPL.O    0.351828
AMZN.O    0.581880
SPY       0.171599
GLD       0.130874
EUR=      0.156568
dtype: float64

In [141]:
(rets.mean() * 252).iplot(kind='bar')

In [142]:
rets.std()  # daily volatilities

CLOSE
AAPL.O    0.012133
AMZN.O    0.013946
SPY       0.006010
GLD       0.006269
EUR=      0.004678
dtype: float64

In [143]:
rets.std() * math.sqrt(252)  # annualized volatilities

CLOSE
AAPL.O    0.192603
AMZN.O    0.221386
SPY       0.095412
GLD       0.099521
EUR=      0.074257
dtype: float64

In [144]:
(rets.std() * math.sqrt(252)).iplot(kind='bar')

In [145]:
[3] * 10

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [146]:
# portfolio statistics

In [147]:
# assume a portfolio composed of all financial instruments with equal weighting.
weights = len(rics) * [1 / len(rics)]
weights

[0.2, 0.2, 0.2, 0.2, 0.2]

In [148]:
# multiply the rows times the cols, dot product, and sum

In [149]:
rets.mean() * 252

CLOSE
AAPL.O    0.351828
AMZN.O    0.581880
SPY       0.171599
GLD       0.130874
EUR=      0.156568
dtype: float64

In [150]:
weights

[0.2, 0.2, 0.2, 0.2, 0.2]

In [151]:
# the expected portfolio return 

In [152]:
(rets.mean() * 252).values

array([0.35182842, 0.58187991, 0.17159877, 0.13087413, 0.15656827])

In [153]:
np.array(weights)

array([0.2, 0.2, 0.2, 0.2, 0.2])

In [154]:
((rets.mean() * 252).values * np.array(weights)).sum()

0.2785498993620757

In [155]:
np.dot(rets.mean() * 252, weights)

0.2785498993620757

In [156]:
(rets.mean() * 252) @ weights

0.2785498993620757

In [157]:
def portfolio_return(symbols, weights):
    return np.dot(rets[symbols].mean() * 252, weights)

In [158]:
portfolio_return(rics, weights)

0.2785498993620757

In [159]:
# accounts for diversification effects
rets.cov()

CLOSE,AAPL.O,AMZN.O,SPY,GLD,EUR=
CLOSE,,,,,
AAPL.O,1.472063e-04,0.000082,4.321598e-05,-0.000004,-7.272906e-07
AMZN.O,8.189007e-05,0.000194,3.738545e-05,-0.000002,-1.002449e-06
SPY,4.321598e-05,0.000037,3.612461e-05,-0.000004,3.318002e-07
GLD,-3.839269e-06,-0.000002,-4.365838e-06,0.000039,1.479673e-05
EUR=,-7.272906e-07,-0.000001,3.318002e-07,0.000015,2.188112e-05


In [160]:
# expected portfolio variance
np.dot(weights, np.dot(rets.cov() * 252, weights))

0.007765942834994757

In [161]:
# expected portfolio volatility
math.sqrt(np.dot(weights, np.dot(rets.cov() * 252, weights)))

0.08812458700609471

In [162]:
def portfolio_volatility(symbols, weights):
    return math.sqrt(np.dot(weights, np.dot(rets[symbols].cov() * 252, weights)))

In [163]:
portfolio_volatility(rics, weights)

0.08812458700609471

In [165]:
# To get started, consider just two financial instruments for which portfolio 
# compositions are simulated that add up to 100% (= 1).

In [166]:
fis = ['AAPL.O', 'AMZN.O']

In [188]:
w = np.random.random((500, len(fis)))  # creating *500* random portfolio compositions ...

In [189]:
# show top 10 normalized portfolio configurations
# take each portfolio, and divide by the sum
w = (w / w.sum(axis=1).reshape(500,1))
w.round(2)[:10]

array([[0.62, 0.38],
       [0.46, 0.54],
       [0.7 , 0.3 ],
       [0.23, 0.77],
       [0.55, 0.45],
       [0.25, 0.75],
       [0.32, 0.68],
       [0.07, 0.93],
       [0.6 , 0.4 ],
       [0.57, 0.43]])

In [193]:
# verify all portfolios add up to 1.0
w.sum(axis=1)[:10]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [194]:
# generate 500 portfolio statistics from the weights
portfolios = [(portfolio_volatility(fis, weights), portfolio_return(fis, weights)) for weights in w]

In [198]:
# first 10 portfolios
portfolios[:10]

[(0.17622792630477305, 0.4381377025142025),
 (0.17992341858420813, 0.4757433758303128),
 (0.17671993129906508, 0.4218474722243719),
 (0.19607105850343384, 0.5296853236011746),
 (0.177082484825362, 0.4552532754796238),
 (0.19436395382245428, 0.525374116605448),
 (0.18842364136823303, 0.5087611884233912),
 (0.2122161524306211, 0.5647552681163581),
 (0.1763856600836879, 0.44459607798205114),
 (0.17679928794788558, 0.4518253772512856)]

In [199]:
portfolio_df = pd.DataFrame(np.array(portfolios), columns=['volatility', 'return'])

In [200]:
portfolio_df.head()

,volatility,return
0,0.176228,0.438138
1,0.179923,0.475743
2,0.176720,0.421847
3,0.196071,0.529685
4,0.177082,0.455253


In [202]:
portfolio_df.iplot(x='volatility', y='return', kind='scatter', mode='markers', color='red')

In [218]:
# find the weights which will yield a min volatility portfolio
index = portfolio_df.volatility.values.argmin()
index

288

In [219]:
portfolio_df.iloc[index]

volatility    0.176220
return        0.436272
Name: 288, dtype: float64

In [220]:
w[index]

array([0.63293797, 0.36706203])

In [221]:
portfolio_volatility(fis, w[index]), portfolio_return(fis, w[index])

(0.17621966804731934, 0.43627158543270356)

In [223]:
# now do the same for all the instruments in the RICS list
# except find the portfolio configuration where
# you yield the largest return, regardless of volatility